# Introduction to Python for Data Engineering

This notebook demonstrates how to read data from a public API, process it to create useful information, and store the result in an SQLite database.

It assumes you have read [this article](https://www.startdataengineering.com/post/python-for-de/#extract--load-read-and-write-data-to-any-system).

**Acknowledgment**: Parts of this notebook were developed with the assistance of ChatGPT, an AI language model by OpenAI, to support instructional design and coding tasks related to data engineering concepts.

In [11]:
# Install required packages
!pip install requests

In [12]:
# Import libraries
import requests
import sqlite3
from datetime import datetime

In [22]:
# Ask user for target currency (e.g., 'zar', 'eur', 'gbp')
target_currency = input("Enter the target currency code (e.g., 'zar', 'eur'): ").lower()

Enter the target currency code (e.g., 'zar', 'eur'): gbp


In [23]:
# Read data from CoinGecko API for USD and target currency
url = f"https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd,{target_currency}"
response = requests.get(url)
data = response.json()

usd_price = data["bitcoin"]["usd"]
if target_currency in data["bitcoin"]:
    target_price = data["bitcoin"][target_currency]
else:
    print(f"Currency '{target_currency}' not found. Defaulting to 0.")
    target_price = 0

timestamp = datetime.now().isoformat()

print(f"Bitcoin price in USD: {usd_price}")
print(f"Bitcoin price in {target_currency.upper()}: {target_price}")


Bitcoin price in USD: 120027
Bitcoin price in GBP: 88435


In [24]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Drop old table if it exists
cursor.execute("DROP TABLE IF EXISTS prices")

# Create table with 4 columns
cursor.execute('''
CREATE TABLE prices (
    timestamp TEXT PRIMARY KEY,
    usd REAL,
    currency TEXT,
    converted REAL
)
''')

# Insert the new row
cursor.execute("INSERT INTO prices VALUES (?, ?, ?, ?)", (timestamp, usd_price, target_currency.upper(), target_price))
conn.commit()
conn.close()

# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()


In [25]:
# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows
rows = cursor.execute("SELECT * FROM prices ORDER BY timestamp DESC LIMIT 5").fetchall()
for row in rows:
    print(row)

conn.close()


('2025-08-13T09:49:10.299322', 120027.0, 'GBP', 88435.0)
